In [214]:
#irisのアヤメデータ


#inport, irisデータの読込
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装
    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue
    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル
    """
    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose

        self.n_support_vectors = 0
        self.index_support_vectors = np.array([])
        self.X_sv = np.array([])
        self.lam_sv = np.array([])
        self.y_sv = np.array([])
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        n_samples = len(X)
        self.lmd = np.random.rand(n_samples)
        self.lmd = self.lmd.reshape(-1,1) #λを縦に並べる。（しない方が処理の行数減るかも）
        y = y.reshape(-1,1)
        
        #print(self.lmd.shape) #(75, 1)
        #print(X.shape) #(75, 4)
        #print(y.shape) #(75, )
        
        #print('self.iter', self.iter)
        #print('self.lr', self.lr)
        #print('self.threshold', self.threshold)

        for _ in range(self.iter):
            
            self.lmd = self.lmd.reshape(-1,1)
            #print("self.lmd.T.shape",self.lmd.T.shape)
            yy = y@y.T #(75, 75)
                #print('y',y)
                #print('y.T',y.T)
                #print('yy',yy)
            #print('yy',yy.shape)
            kx = self._kernel_function(X, X) #(75, 75)
            #print('kx',kx.shape)
            tmp2 = yy * kx #(75, 75) ・・・アダマール積でOK？（参考シートは＠）
            #print('tmp2',tmp2.shape)
            tmp3 = np.sum(self.lmd.T @ tmp2, axis=0) 
            #print('tmp3',tmp3.shape)
            #print('tmp3',tmp3)
            tmp = 1 -  tmp3
            #print('tmp',tmp.shape) #()
            #print('tmp',tmp)
            #print("check lmd.shape", self.lmd)
            tmp = tmp.reshape(-1,1)  
            self.lmd = self.lmd + self.lr *tmp
            #self.lmd = self.lmd.reshape(-1,1)
            #print('self.lmd', self.lmd)

            #print("self.lmd.shape",self.lmd.shape)
            self.lmd = self.lmd.flatten()
            for i in range(n_samples):
                if self.lmd[i] < 0:
                    self.lmd[i] = 0
                
        for i in range(n_samples):
            #閾値より大きい？小さい？　→　要確認
            if self.lmd[i] > self.threshold:
                #サポートベクトルの数を記録
                self.n_support_vectors += 1
                #サポートベクトルのインデックス
                self.index_support_vectors = np.append(self.index_support_vectors, i)
                #サポートベクトルの特徴量
                if self.n_support_vectors == 1:
                    self.X_sv =  X[i, :]
                else:
                    self.X_sv = np.vstack((self.X_sv, X[i, :]))
                #サポートベクトルの未定乗数を記録
                self.lam_sv = np.append(self.lam_sv, self.lmd[i])
               #サポートベクトルのラベルを記録
                self.y_sv  = np.append(self.y_sv, y[i]) 

        print("index",self.index_support_vectors.shape)
        print("数量",self.n_support_vectors)
        print("特徴量",self.X_sv.shape)
        print("未定乗数",self.lam_sv.shape)
        print("未定乗数",self.lam_sv)
        print("ラベル", self.y_sv.shape)
             
         
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
            

            
    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """

        #fx = np.sum(lamda_sv[0])*y(lamda_sv)@k(x, sn)
        
        print('self.lam_sv',self.lam_sv)
        print('self.X_sv',self.lam_sv)
        print('self.y_sv',self.y_sv)
        print('X',X)
        
        tmp = np.sum(self.lam_sv) 
        k = self._kernel_function(X, self.X_sv)

        fx = tmp * self.y_sv @k
        
        print('fx',fx)
        print('fx.shape',fx.shape)
        
        output = np.where(fx <  0, -1, 1)
        print('output',output)
        
        return output
    
    def _kernel_function(self, X1,X2):
        return X2 @ X1.T


#分析データの取得
iris_dataset = load_iris()
X = pd.DataFrame(iris_dataset.data, columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
y = pd.DataFrame(iris_dataset.target, columns=["Species"])
df = pd.concat([X, y], axis=1)
X = df.query("Species == 1 | Species == 2").loc[:,["sepal_length", "sepal_width", "petal_length", "petal_width"]].values
y = df.query("Species == 1 | Species == 2").loc[:,"Species"].values

#ラベルの-1、１化
y = np.where(y ==  2, -1, 1)

#train, testデータへの分割
X_train, X_test, y_train, y_test = train_test_split(X,y)

#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)
    
#機械学習(fit)

num_iter = 10000
lr = 0.0001

svm = ScratchSVMClassifier(num_iter, lr, kernel='linear', threshold=1e-5, verbose=False)
svm.fit(X_train_transformed, y_train, X_test_transformed, y_test)

output = svm.predict(X_test_transformed)

print('output',output)
print('y_test',y_test)


index (17,)
数量 17
特徴量 (17, 4)
未定乗数 (17,)
未定乗数 [1.04806754 1.47070839 0.01633462 1.06591066 0.30494648 0.67646026
 0.07314428 0.78497241 0.04729437 2.08389353 1.32954177 0.01674808
 0.09083396 0.18981622 0.21213623 0.1516665  1.05443456]
ラベル (17,)
self.lam_sv [1.04806754 1.47070839 0.01633462 1.06591066 0.30494648 0.67646026
 0.07314428 0.78497241 0.04729437 2.08389353 1.32954177 0.01674808
 0.09083396 0.18981622 0.21213623 0.1516665  1.05443456]
self.X_sv [1.04806754 1.47070839 0.01633462 1.06591066 0.30494648 0.67646026
 0.07314428 0.78497241 0.04729437 2.08389353 1.32954177 0.01674808
 0.09083396 0.18981622 0.21213623 0.1516665  1.05443456]
self.y_sv [-1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1.]
X [[ 0.69825592  0.78629632  0.85330043  1.77224003]
 [ 0.06919653 -1.15784294 -0.04087667 -0.43082078]
 [-0.40259801  0.46227311 -0.16861625  0.30353282]
 [ 0.69825592  0.46227311  0.08686292  0.05874829]
 [ 1.01278562  0.78629632  0.2146025   1.52745549]
 [-2.1325113

In [281]:
import numpy as np
from matplotlib import pyplot
%matplotlib inline
import sys


def f(x, y):
    return x - y


param = sys.argv
print(param)

numpy.random.seed()
N = 30 #サンプル数
d = 2
X = numpy.random.randn(N, d)
#print(X.shape) → (30, 2)
#print(X)

T = numpy.array([1 if f(x, y) > 0 else - 1 for x, y in X])
#print(T) → 1か-1が入ってる
#print(T.shape) → (30, )

alpha = numpy.zeros(N)
#print(alpha.shape) → (30,)
beta = 1.0
eta_al = 0.0001 # update ratio of alpha
eta_be = 0.1 # update ratio of beta
itr = 1000

for _itr in range(itr):
    for i in range(N):
        #delta = 1 - (T[i] * X[i]).dot(alpha * T * X.T).sum() - beta * T[i] * alpha.dot(T)
        tmp1 = (T[i] * X[i])
        tmp2 = (alpha * T * X.T)
        #print(tmp1.shape) → (2, )
        #print(tmp2.shape) →(2,30)
        tmp3 = tmp1 @ tmp2
        #print(tmp3.shape) →(30, )
        
        #delta = 1 - (tmp1).dot(tmp2).sum()   - beta * T[i] * alpha.dot(T)
        #delta = 1 -  tmp3.sum()   - beta * T[i] * alpha.dot(T)
        #delta = 1 -  np.sum(tmp3)   - beta * T[i] * alpha.dot(T)
        #delta = 1 -  np.sum((T[i] * X[i]) @ (alpha * T * X.T))   - beta * T[i] * alpha.dot(T)
        delta = 1 -  np.sum((T[i] * X[i]) @ (alpha * T * X.T))   - beta * T[i] * alpha.dot(T)
        #delta = 1 -  (T[i] * X[i]) * np.sum((alpha * T * X.T))   - beta * T[i] * alpha.dot(T)

        #print(type(delta)) → float
        #delta = 1 - (T[i] * X[i]) * np.sum(alpha * T * X.T) - beta * T[i] * alpha.dot(T)
        #delta = 1 - (T[i] * X[i]).dot(np.sum(alpha * T * X.T)) - beta * T[i] * alpha.dot(T)
        alpha[i] += eta_al * delta
    for i in range(N):
        beta += eta_be * alpha.dot(T) ** 2 / 2

index = alpha > 0
w = (alpha * T).T.dot(X)
b = (T[index] - X[index].dot(w)).mean()

if '-d' in param or '-s' in param:
    seq = numpy.arange(-3, 3, 0.02)
    pyplot.figure(figsize = (6, 6))
    pyplot.xlim(-3, 3)
    pyplot.ylim(-3, 3)
    pyplot.plot(seq, -(w[0] * seq + b) / w[1], 'k-')
    pyplot.plot(X[T ==  1,0], X[T ==  1,1], 'ro')
    pyplot.plot(X[T == -1,0], X[T == -1,1], 'bo')

    if '-s' in param:
        pyplot.savefig('graph.png')

    if '-d' in param:
        pyplot.show()

['/Users/tomohisamatsuda/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/Users/tomohisamatsuda/Library/Jupyter/runtime/kernel-e38bd27f-25ca-45e7-ab6e-f2c14f5ddb0e.json']


### 【問題1】ラグランジュの未定乗数法による最急降下
### 【問題2】サポートベクターの決定
### 【問題3】推定

In [320]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

class ScratchSVMClassifier():

    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose

        self.n_support_vectors = 0
        self.index_support_vectors = np.array([])
        self.X_sv = np.array([])
        self.lam_sv = np.array([])
        self.y_sv = np.array([])
        
    def fit(self, X, y, X_val=None, y_val=None):
        #λの初期値をランダムに設定
        n_samples = X.shape[0]
        lmd = np.random.rand(n_samples)
        
        #λ、yは縦一列に並べることにする。
        lmd = lmd.reshape(-1,1) 
        y = y.reshape(-1,1)

        #最急降下法をイテレーション回数実施
        for _ in range(self.iter):
            
            #ラグランジュの未定乗数法による最急降下法の計算
            yy = y@y.T
            kx = self._kernel_function(X, X) 
            yykx = yy * kx
            sigma = np.sum(lmd.T @ yykx, axis=0) 
            sigma = sigma.reshape(-1,1)  #λを縦一列に並べているのでshapeを合わせる。
            lmd = lmd + self.lr * (1- sigma)
            lmd = np.where(lmd <  0, 0, lmd) #λ>0とする。
 
        #サポートベクトルの情報を保管する。
        for i in range(n_samples):
            if lmd[i] > self.threshold:
                
                #サポートベクトルの数
                self.n_support_vectors += 1
                
                #サポートベクトルのインデックス
                self.index_support_vectors = np.append(self.index_support_vectors, i)

                #サポートベクトルの特徴量
                if self.n_support_vectors == 1:
                    self.X_sv =  X[i, :]
                else:
                    self.X_sv = np.vstack((self.X_sv, X[i, :]))

                #サポートベクトルのラグランジュ未定乗数
                self.lam_sv = np.append(self.lam_sv, lmd[i])

                #サポートベクトルのラベル
                self.y_sv  = np.append(self.y_sv, y[i]) 

        print("index",self.index_support_vectors)
        print("サポートベクトルの個数",self.n_support_vectors)
        #print("特徴量",self.X_sv.shape)
        #print("未定乗数",self.lam_sv.shape)
        #print("未定乗数",self.lam_sv)
        #print("ラベル", self.y_sv.shape)
             
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()

    #推定
    def predict(self, X):
        
        #推定式の計算
        tmp = np.sum(self.lam_sv) 
        k = self._kernel_function(X, self.X_sv)
        fx = tmp * self.y_sv @k
        
        #fx計算結果により出力を分類
        output = np.where(fx <  0, -1, 1)        
        return output
    
    #カーネル関数の計算メソッド
    def _kernel_function(self, X1,X2):
        return X2 @ X1.T

#シンプルデータセット１の作成
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int) #f0に1, f1に-1を対応させている。
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index] 
y = y[random_index]

#train, testデータへの分割
X_train, X_test, y_train, y_test = train_test_split(X,y)

#標準化(Xのみ標準化。yはしない。)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

#機械学習(fit)
num_iter = 1000
lr = 0.001
thre = 1e-5
svm = ScratchSVMClassifier(num_iter, lr, kernel='linear', threshold=thre, verbose=False)
svm.fit(X_train_transformed, y_train, X_test_transformed, y_test)


index [ 11.  48.  87.  89. 103. 112. 160. 162. 167. 194. 261. 294. 335.]
サポートベクトルの個数 13


### 【問題4】学習と推定

In [277]:
#スクラッチによる推定
output = svm.predict(X_test_transformed)
acc = accuracy_score(output, y_test)
pre = precision_score(output, y_test)
rec = recall_score(output, y_test)
f1 =   f1_score(output, y_test)
print("スクラッチによる推定\n acc: {}  pre:{}  rec:{}  f1:{}".format(acc, pre, rec, f1))

#scikit-learnによる推定
svc = SVC()
svc.fit(X_train_transformed, y_train)
y_pred = svc.predict(X_test_transformed)
acc = accuracy_score(y_pred, y_test)
pre = precision_score(y_pred, y_test)
rec = recall_score(y_pred, y_test)
f1 =   f1_score(y_pred, y_test)
print("scikit-learnによる推定\n acc: {}  pre:{}  rec:{}  f1:{}".format(acc, pre, rec, f1))

スクラッチによる推定
 acc: 1.0  pre:1.0  rec:1.0  f1:1.0
scikit-learnによる推定
 acc: 1.0  pre:1.0  rec:1.0  f1:1.0


### 【問題5】決定領域の可視化

In [325]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['1', '-1']):
    
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    
    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = svm.predict(mesh).reshape(mesh_f0.shape)
    #↑  = model.pre〜のままだと動かないので、model→svmへ書き換えた。

    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')

    #サポートベクトルの表示を追加
    plt.scatter(svm.X_sv[:, 0], svm.X_sv[:, 1], color='yellow', label="SV", marker='o')

    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()

y_train = y_train.flatten() 
decision_region(X_train_transformed, y_train, svm.fit(X_train_transformed, y_train))

index [ 11.  48.  87.  89. 103. 112. 160. 162. 167. 194. 261. 294. 335.   5.
  48.  89. 103. 112. 160. 162. 167. 194. 207. 261. 272. 294. 309. 335.
   0.  76.  81.  89. 103. 112. 160. 162. 167. 194. 261. 272. 294. 335.
   0.  48.  89. 103. 112. 160. 162. 167. 194. 261. 272. 335.  81.  89.
 103. 112. 151. 160. 162. 167. 194. 261. 270. 309. 335.  76.  89. 103.
 112. 160. 162. 167. 218. 261. 272. 335.]
サポートベクトルの個数 65


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 65 is different from 78)

In [302]:
#https://enjoyworks.jp/tech-blog/3087

def scat(X,y):
    plt.figure(figsize=(6, 4))
    plt.scatter(X[y==-1][:,0],X[y==-1][:,1],label="-1")
    plt.scatter(X[y==1][:,0],X[y==1][:,1],label="1")

    
    plt.legend()
    X0_min,X0_max = X[:,0].min(),X[:,0].max()
    X1_min,X1_max = X[:,1].min(),X[:,1].max()
    XX0,XX1 = np.meshgrid(np.linspace(X0_min,X0_max),np.linspace(X1_min,X1_max))
    grid = np.c_[XX0.ravel(),XX1.ravel()]

    probs = svm.predict_proba(grid).reshape(XX0.shape)

    plt.xlim(-3,3)
    plt.ylim(-3,3)
    plt.contour(XX0,XX1,probs,[0.5],linewidths=2)
    

    plt.scatter(X[svm.ind[0],0],X[svm.ind[0],1],s=200,marker='o',color=(0,0,0,0),edgecolor='k')

In [324]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['1', '-1']):

        
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = svm.predict(mesh).reshape(mesh_f0.shape)
    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()

y_train = y_train.flatten() 
decision_region(X_train_transformed, y_train, svm.fit(X_train_transformed, y_train))

index [ 11.  48.  87.  89. 103. 112. 160. 162. 167. 194. 261. 294. 335.   5.
  48.  89. 103. 112. 160. 162. 167. 194. 207. 261. 272. 294. 309. 335.
   0.  76.  81.  89. 103. 112. 160. 162. 167. 194. 261. 272. 294. 335.
   0.  48.  89. 103. 112. 160. 162. 167. 194. 261. 272. 335.  81.  89.
 103. 112. 151. 160. 162. 167. 194. 261. 270. 309. 335.]
サポートベクトルの個数 54


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 54 is different from 67)

In [326]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['setosa', 'versicolor']):
    
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = model.predict(mesh).reshape(mesh_f0.shape)
    # plot
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    plt.scatter(X[svm.ind[0],0],X[svm.ind[0],1], s=80, color='yellow', label='SV', marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()  

y_train = y_train.flatten() 
decision_region(X_train_transformed, y_train, svm.fit(X_train_transformed, y_train))
    